# LangChain with networkx

Date: 2024/08/09

Reference: https://python.langchain.com/v0.1/docs/integrations/graphs/networkx/

In [1]:
from langchain.indexes import GraphIndexCreator
from langchain_openai import OpenAI

In [7]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [3]:
#with open("doc/state_of_the_union.txt") as f:
#    all_text = f.read()

In [16]:
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("doc/n1410000.pdf")
data = loader.load()
print(data[0].page_content)

第4章
第1 節 AIの進化に伴う課題と現状の取組
進化してきたAIは我々の生活に便利さをもたらす一方で、活用に当たっては留意すべきリスク
や課題も存在している。これまで、AI全般についても、不適切なデータや偏ったデータを学習に
使用することでモデルのバイアスや誤差が増加し、予測の信頼性が低下する点や、多くの従来の機
械学習モデルについてブラックボックス（透明性の欠如）となっていてその内部動作が理解しにく
く、重要な意思決定の場面で問題を引き起こす可能性が指摘されていた。これに加え、生成AIが
爆発的に発展・普及する中で、特有の課題・リスクも明らかになってきた。以下に生成AIが抱え
るリスク・課題を技術的/社会・経済的な観点から概観する。
	1 
生成AIが抱える課題
2024年4月に総務省・経済産業省が策定した「AI事業者ガイドライン（第1.0版）
」では、
（従
来から存在する）AIによるリスクに加えて、生成AIによって顕在化したリスクについて例示して
いる（図表Ⅰ-4-1-1）
。例えば、従来から存在するAIによるリスクとして、バイアスのある結果及
び差別的な結果が出力されてしまう、フィルターバブル及びエコーチェンバー現象
＊1が生じてしま
う、データ汚染攻撃のリスク（AIの学習実施時の性能劣化及び誤分類につながるような学習データ
の混入等）
、AIの利用拡大に伴う計算リソースの拡大によるエネルギー使用量及び環境負荷
＊2等が
挙げられている。また、生成AIによって顕在化したリスクとしては、ハルシネーション等が挙げら
れる。生成AIは事実に基づかない誤った情報をもっともらしく生成することがあり、これをハルシ
ネーション（幻覚）と呼ぶ。技術的な対策が検討されているものの完全に抑制できるものではない
ため、生成AIを活用する際には、ハルシネーションが起こる可能性を念頭に置き、検索を併用する
など、ユーザーは生成AIの出力した答えが正しいかどうかを確認することが望ましい。また、生成
AIの利用において、個人情報や機密情報がプロンプトとして入力され、そのAIからの出力等を通
じて流出してしまうリスクや、ディープフェイクによる偽画像及び偽動画といった偽・誤情報を鵜
呑みにしてしまい、情報操作や世論工作に使われるといったリスク、既存の情報に基づいてAIによ
り生成された回

In [4]:
text = "\n".join(all_text.split("\n\n")[105:108])
text

'It won’t look like much, but if you stop and look closely, you’ll see a “Field of dreams,” the ground on which America’s future will be built. \nThis is where Intel, the American company that helped build Silicon Valley, is going to build its $20 billion semiconductor “mega site”. \nUp to eight state-of-the-art factories in one place. 10,000 new good-paying jobs. '

In [5]:
graph = index_creator.from_text(text)

In [6]:
graph.get_triples()

[('Intel', '$20 billion semiconductor “mega site”', 'is going to build'),
 ('Intel', 'Silicon Valley', 'helped build'),
 ('Intel', 'up to eight state-of-the-art factories', 'is going to build'),
 ('Intel', '10,000 new good-paying jobs', 'is going to create'),
 ('America', '“Field of dreams”', 'will be built on')]

In [7]:
from langchain.chains import GraphQAChain

In [8]:
chain = GraphQAChain.from_llm(OpenAI(temperature=0), graph=graph, verbose=True)

In [9]:
chain.run("what is Intel going to build?")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphQAChain chain...
Entities Extracted:
 Intel
Full Context:
Intel is going to build $20 billion semiconductor “mega site”
Intel helped build Silicon Valley
Intel is going to build up to eight state-of-the-art factories
Intel is going to create 10,000 new good-paying jobs

> Finished chain.


' Intel is going to build a $20 billion semiconductor "mega site" and up to eight state-of-the-art factories.'